# Bedrock Agents

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to use the `bedrock-agent` and the `bedrock-agent-runtime` boto3 clients to:
- create an pool & silo agent 
- create and action group for pool & silo agent
- associate the pool & silo agent with the action group and prepare the agents
- create an agent alias for pool & silo agent
- invoke the pool & silo agent

We will use Bedrock's Claude v2.1 using the Boto3 API. 

**Note:** *Running through this notebook will incur you AWS cost through the provisioned resources and through interaction with LLM.*

#### Pre-requisites
This notebook requires permissions to: 
- create and delete Amazon IAM roles
- create, update and invoke AWS Lambda functions 
- create, update and delete Amazon S3 buckets 
- access Amazon Bedrock 

If you are running this notebook without an Admin role, make sure that your role include the following managed policies:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonS3FullAccess
- AmazonDynamoDBFullAccess
- AmazonBedrockFullAccess



## Notebook setup
Before starting, let's import the required packages and configure the support variables

In [ ]:
! pip install --upgrade boto3
! pip show boto3

In [ ]:
import logging
import boto3
import random
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint

In [ ]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
dynamodb_client = boto3.client('dynamodb')

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

In [ ]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
pool_agent_name = "pool-app-svcs-agent"
pool_agent_alias_name = "pool-agent-alias"
silo_agent_name = "silo-app-svcs-agent"
silo_agent_alias_name = "silo-agent-alias"


bucket_name = f'{pool_agent_name}-{suffix}'
bucket_key = f'{pool_agent_name}-schema.json'
schema_name = 'app_services_agent_openapi_schema.json'
schema_arn = f'arn:aws:s3:::{bucket_name}/{bucket_key}'
bedrock_agent_bedrock_allow_policy_name = f"agents-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"agents-s3-allow-{suffix}"


pool_agent_lambda_role_name = f'{pool_agent_name}-lambda-role-{suffix}'
pool_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_pool_svcs'
pool_agent_lambda_code_path = "pool_agent_lambda_function.py"
pool_agent_lambda_name = f'{pool_agent_name}-{suffix}'

silo_agent_lambda_role_name = f'{silo_agent_name}-lambda-role-{suffix}'
silo_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_silo_svcs'
silo_agent_lambda_code_path = "silo_agent_lambda_function.py"
silo_agent_lambda_name = f'{silo_agent_name}-{suffix}'

### Create S3 bucket and upload API Schema

Agents require an API Schema stored on s3. Let's create an S3 bucket to store the file and upload the file to the newly created bucket

In [ ]:
# Create S3 bucket for Open API schema
s3bucket = s3_client.create_bucket(
    Bucket=bucket_name
)

In [ ]:
# Upload Open API schema to this s3 bucket
s3_client.upload_file(schema_name, bucket_name, bucket_key)

# Pool Agent

### Create pool DynamoDB tables

In [ ]:
pool_product_table = 'pool-product-table'
pool_order_table = 'pool-order-table'


pool_product_table_schema = [
    {
        'AttributeName': 'tenantId',  
        'AttributeType': 'S',      
    },
    {
        'AttributeName': 'productId',  
        'AttributeType': 'S',           
    },
]

pool_order_table_schema = [
    {
        'AttributeName': 'tenantId',  
        'AttributeType': 'S',      
    },
    {
        'AttributeName': 'orderId',  
        'AttributeType': 'S',           
    },
]


pool_product_key_schema = [
    {
        'AttributeName': 'tenantId',  
        'KeyType': 'HASH',         
    },
    {
        'AttributeName': 'productId', 
        'KeyType': 'RANGE',            
    }
]

pool_order_key_schema = [
    {
        'AttributeName': 'tenantId',  
        'KeyType': 'HASH',         
    },
    {
        'AttributeName': 'orderId', 
        'KeyType': 'RANGE',            
    }
]


provisioned_throughput = {
    'ReadCapacityUnits': 5,   
    'WriteCapacityUnits': 5   
}

pool_product_table_response = dynamodb_client.create_table(
    TableName=pool_product_table,
    AttributeDefinitions=pool_product_table_schema,
    KeySchema=pool_product_key_schema,
    ProvisionedThroughput=provisioned_throughput
)

pool_product_table_arn = pool_product_table_response['TableDescription']['TableArn']

print(f"Table {pool_product_table} created. Status: {pool_product_table_response['TableDescription']['TableArn']}")

pool_order_table_response = dynamodb_client.create_table(
    TableName=pool_order_table,
    AttributeDefinitions=pool_order_table_schema,
    KeySchema=pool_order_key_schema,
    ProvisionedThroughput=provisioned_throughput
)

pool_order_table_arn = pool_order_table_response['TableDescription']['TableArn']

print(f"Table {pool_order_table} created. Status: {pool_order_table_response['TableDescription']['TableArn']}")


### Create ABAC role - assumed in runtime

In [ ]:
try:
    app_assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": [
                    sts_client.get_caller_identity()["Arn"]
                ]
            },
            "Action": [
                "sts:AssumeRole",
                "sts:TagSession"
            ],
            "Condition": {
                "StringLike": {
                    "aws:RequestTag/TenantId": "*"
                }
            }
        }
    ]
}

    app_assume_role_policy_document_json = json.dumps(app_assume_role_policy_document)

    app_lambda_assume_role = iam_client.create_role(
        RoleName='app-assume-role',
        AssumeRolePolicyDocument=app_assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    app_lambda_assume_role = iam_client.get_role(RoleName='app-assume-role')

    
    
iam_client.put_role_policy(
    RoleName='app-assume-role',
    PolicyName='app-policy',
    PolicyDocument='{"Version": "2012-10-17","Statement": [{"Sid": "VisualEditor0","Effect": "Allow","Action": ["dynamodb:PutItem","dynamodb:GetItem","dynamodb:Query"],"Resource": ["'+pool_product_table_arn+'","'+pool_order_table_arn+'"],"Condition": {"ForAllValues:StringEquals": {"dynamodb:LeadingKeys": ["${aws:PrincipalTag/TenantId}"]}}}]}'
)


### Create Lambda function for Action Group
Let's now create the lambda function required by the agent action group. We first need to create the lambda IAM role and it's policy. After that, we package the lambda function into a ZIP format to create the function

In [ ]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    pool_agent_lambda_iam_role = iam_client.create_role(
        RoleName=pool_agent_lambda_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    pool_agent_lambda_iam_role = iam_client.get_role(RoleName=pool_agent_lambda_role_name)

iam_client.attach_role_policy(
    RoleName=pool_agent_lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)


In [ ]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(pool_agent_lambda_code_path)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
pool_lambda_function = lambda_client.create_function(
    FunctionName=pool_agent_lambda_name,
    Runtime='python3.12',
    Timeout=180,
    Role=pool_agent_lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='pool_agent_lambda_function.lambda_handler'
)

### Create Agent
We will now create our agent. To do so, we first need to create the agent policies that allow bedrock model invocation  and s3 bucket access. 

In [ ]:
# Create IAM policies for agent

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/anthropic.claude-v2:1"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [ ]:
bedrock_agent_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAgentAccessOpenAPISchema",
            "Effect": "Allow",
            "Action": ["s3:GetObject"],
            "Resource": [
                schema_arn
            ]
        }
    ]
}


bedrock_agent_s3_json = json.dumps(bedrock_agent_s3_allow_policy_statement)
agent_s3_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_s3_allow_policy_name,
    Description=f"Policy to allow invoke Lambda that was provisioned for it.",
    PolicyDocument=bedrock_agent_s3_json
)

In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
pool_agent_role = iam_client.create_role(
    RoleName=pool_agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=pool_agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=pool_agent_role_name,
    PolicyArn=agent_s3_schema_policy['Policy']['Arn']
)

#### Creating Pool Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [ ]:
# Create Agent


pool_agent_instruction = """
You are an agent that helps customers to create products and orders. You can
1/create a product with provided product name and product price
2/get all products details
3/get a product detail using product id provided by the customer
4/create a order for a product using product id and quantity provided by the customer
5/get all orders details
If no clear instruction are provided, if an customer asks about your functionality, provide guidance in natural language 
and do not include function names on the output.
"""

response = bedrock_agent_client.create_agent(
    agentName=pool_agent_name,
    agentResourceRoleArn=pool_agent_role['Role']['Arn'],
    description="Agent for handling app requests.",
    idleSessionTTLInSeconds=1800,
    foundationModel="anthropic.claude-v2:1",
    instruction=pool_agent_instruction,
)

Looking at the created agent, we can see its status and agent id

In [ ]:
response

Let's now store the agent id in a local variable to use it on the next steps

In [ ]:
pool_agent_id = response['agent']['agentId']
pool_agent_id

### Create Agent Action Group
We will now create and agent action group that uses the lambda function and API schema files created before.
The `create_agent_action_group` function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet create an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In [ ]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
pool_agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=pool_agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': pool_lambda_function['FunctionArn']
    },
    actionGroupName='PoolAppServicesActionGroup', 
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description='Actions for creating product, get products, get a product, create order and get orders'
)

In [ ]:
pool_agent_action_group_response

### Allowing Agent to invoke Action Group Lambda
Before using our action group, we need to allow our agent to invoke the lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [ ]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=pool_agent_lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{pool_agent_id}",
)

### Preparing Agent
Let's create a DRAFT version of the agent that can be used for internal testing.

In [ ]:
pool_agent_prepare = bedrock_agent_client.prepare_agent(agentId=pool_agent_id)
pool_agent_prepare

### Create Agent alias
We will now create an alias of the agent that can be used to deploy the agent.

In [ ]:
# Pause to make sure agent is prepared
time.sleep(30)
pool_agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=pool_agent_id,
    agentAliasName=pool_agent_alias_name
)

In [ ]:
pool_agent_alias

In [ ]:
def get_temporary_credentials(tenantId):
    
    app_lambda_assume_role = iam_client.get_role(RoleName='app-assume-role')

    assume_role_response = sts_client.assume_role(
            RoleArn=app_lambda_assume_role['Role']['Arn'],
            DurationSeconds=900,
            RoleSessionName="tenant",
            Tags=[{"Key": "TenantId", "Value": tenantId}]
    )


    return {
    
        'accessKeyId': assume_role_response["Credentials"]["AccessKeyId"],
        'secretAccessKey': assume_role_response["Credentials"]["SecretAccessKey"],
        'sessionToken': assume_role_response["Credentials"]["SessionToken"]

    }

In [ ]:
# Method to capture agent response and metrics
def capture_metrics(agentResponse, tenantId):
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                trace_json = event['trace']['trace']
                stage =''
                # uncomment below line to get detailed tracing messages
                # logger.info(json.dumps(event['trace'], indent=2))
                if 'preProcessingTrace' in trace_json:
                    trace_json = trace_json['preProcessingTrace']
                    stage = 'preProcessingTrace'
                elif 'orchestrationTrace' in trace_json:
                    trace_json = trace_json['orchestrationTrace']
                    stage = 'orchestrationTrace'
                elif 'postProcessingTrace' in trace_json:
                    trace_json = trace_json['postProcessingTrace']
                    stage = 'postProcessingTrace'



                if  'modelInvocationOutput' in trace_json and \
                    'metadata' in trace_json["modelInvocationOutput"] and \
                    "usage" in trace_json["modelInvocationOutput"]["metadata"] :
                    print(f'stage: {stage} tenantId: {tenantId} inputTokens: {trace_json["modelInvocationOutput"]["metadata"]["usage"]["inputTokens"]}')
                    print(f'stage: {stage} tenantId: {tenantId} outputTokens: {trace_json["modelInvocationOutput"]["metadata"]["usage"]["outputTokens"]}')
                # else:
                    # print("usage attribute is not present.")


            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

## Invoke Agent
Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [ ]:
def invoke_pool_agent(tenantId, task_input_text):
    pool_agent_alias_id = pool_agent_alias['agentAlias']['agentAliasId']

    ## create a random id for session initiator id
    session_id:str = str(uuid.uuid1())
    enable_trace:bool = True
    end_session:bool = False
    
    creds=get_temporary_credentials(tenantId)
    pool_agentResponse = bedrock_agent_runtime_client.invoke_agent(
    sessionState={
        'sessionAttributes': {
            'tenantId': tenantId,
            'accessKeyId': creds['accessKeyId'],
            'secretAccessKey': creds['secretAccessKey'],
            'sessionToken': creds['sessionToken']
            
            
        }
    },
    inputText=task_input_text,
    agentId=pool_agent_id,
    agentAliasId=pool_agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
    )

    logger.info(pprint.pprint(pool_agentResponse))
    capture_metrics(pool_agentResponse, tenantId)
    

### Create data for tenant1

In [ ]:
# invoke the agent API
tenantId='tenant1'
task_input_text = 'create two products product1 and product2 with prices 10 and 20 respectively' 
invoke_pool_agent(tenantId, task_input_text)

### Create data for tenant2

In [ ]:
tenantId='tenant2'
task_input_text = 'create two products product3 and product4 with prices 30 and 40 respectively'
invoke_pool_agent(tenantId, task_input_text)

### Get all products of tenant1

In [ ]:
tenantId='tenant1'
task_input_text = 'get all products' 
invoke_pool_agent(tenantId, task_input_text)

In [ ]:
# %%time
# event_stream = agentResponse['completion']
# try:
#     for event in event_stream:        
#         if 'chunk' in event:
#             data = event['chunk']['bytes']
#             logger.info(f"Final answer ->\n{data.decode('utf8')}")
#             agent_answer = data.decode('utf8')
#             end_event_received = True
#             # End event indicates that the request finished successfully
#         elif 'trace' in event:
#             logger.info(json.dumps(event['trace'], indent=2))
#         else:
#             raise Exception("unexpected event.", event)
# except Exception as e:
#     raise Exception("unexpected event.", e)

# Silo Agent

### Create Silo DynamoDB tables

In [ ]:
silo_product_table = 'silo-product-table'
silo_order_table = 'silo-order-table'


silo_product_table_schema = [
    {
        'AttributeName': 'tenantId',  
        'AttributeType': 'S',      
    },
    {
        'AttributeName': 'productId',  
        'AttributeType': 'S',           
    },
]

silo_order_table_schema = [
    {
        'AttributeName': 'tenantId',  
        'AttributeType': 'S',      
    },
    {
        'AttributeName': 'orderId',  
        'AttributeType': 'S',           
    },
]


silo_product_key_schema = [
    {
        'AttributeName': 'tenantId',  
        'KeyType': 'HASH',         
    },
    {
        'AttributeName': 'productId', 
        'KeyType': 'RANGE',            
    }
]

silo_order_key_schema = [
    {
        'AttributeName': 'tenantId',  
        'KeyType': 'HASH',         
    },
    {
        'AttributeName': 'orderId', 
        'KeyType': 'RANGE',            
    }
]


provisioned_throughput = {
    'ReadCapacityUnits': 5,   
    'WriteCapacityUnits': 5   
}

silo_product_table_response = dynamodb_client.create_table(
    TableName=silo_product_table,
    AttributeDefinitions=silo_product_table_schema,
    KeySchema=silo_product_key_schema,
    ProvisionedThroughput=provisioned_throughput
)

silo_product_table_arn = silo_product_table_response['TableDescription']['TableArn']

print(f"Table {silo_product_table} created. Status: {silo_product_table_response['TableDescription']['TableArn']}")

silo_order_table_response = dynamodb_client.create_table(
    TableName=silo_order_table,
    AttributeDefinitions=silo_order_table_schema,
    KeySchema=silo_order_key_schema,
    ProvisionedThroughput=provisioned_throughput
)

silo_order_table_arn = silo_order_table_response['TableDescription']['TableArn']

print(f"Table {silo_order_table} created. Status: {silo_order_table_response['TableDescription']['TableArn']}")


### Create Silo Lambda function for Action Group

In [ ]:
# Create IAM Role for the Lambda function
try:
    silo_assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    silo_assume_role_policy_document_json = json.dumps(silo_assume_role_policy_document)

    silo_agent_lambda_iam_role = iam_client.create_role(
        RoleName=silo_agent_lambda_role_name,
        AssumeRolePolicyDocument=silo_assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    silo_agent_lambda_iam_role = iam_client.get_role(RoleName=silo_agent_lambda_role_name)

iam_client.attach_role_policy(
    RoleName=silo_agent_lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

dynamodb_policies_statement = {
    "Version": "2012-10-17",
    "Statement": [
    {
        "Sid": "QueryDynamoDB",
        "Effect": "Allow",
        "Action": [
            "dynamodb:PutItem",
            "dynamodb:GetItem",
            "dynamodb:Query"
        ],
        "Resource": [
            f"arn:aws:dynamodb:{region}:{account_id}:table/silo-product-table",
            f"arn:aws:dynamodb:{region}:{account_id}:table/silo-order-table"
            
        ]
    }
  ]
}

silo_dynamodb_policy = iam_client.create_policy(
    PolicyName='silo_dynamodb_policy',
    PolicyDocument=json.dumps(dynamodb_policies_statement)
)

time.sleep(10)

iam_client.attach_role_policy(
    RoleName=silo_agent_lambda_role_name,
    PolicyArn=silo_dynamodb_policy['Policy']['Arn']
)

In [ ]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(silo_agent_lambda_code_path)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
silo_lambda_function = lambda_client.create_function(
    FunctionName=silo_agent_lambda_name,
    Runtime='python3.12',
    Timeout=180,
    Role=silo_agent_lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='silo_agent_lambda_function.lambda_handler'
)

### Create Agent

In [ ]:
silo_assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

silo_agent_role = iam_client.create_role(
    RoleName=silo_agent_role_name,
    AssumeRolePolicyDocument=json.dumps(silo_assume_role_policy_document)
)


time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=silo_agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=silo_agent_role_name,
    PolicyArn=agent_s3_schema_policy['Policy']['Arn']
)

#### Creating Silo Agent

In [ ]:
# Create Agent


silo_agent_instruction = """
You are an agent that helps customers to create products and orders. You can
1/create a product with provided product name and product price
2/get all products details
3/get a product detail using product id provided by the customer
4/create a order for a product using product id and quantity provided by the customer
5/get all orders details
If no clear instruction are provided, if an customer asks about your functionality, provide guidance in natural language 
and do not include function names on the output.
"""

silo_agent_response = bedrock_agent_client.create_agent(
    agentName=silo_agent_name,
    agentResourceRoleArn=silo_agent_role['Role']['Arn'],
    description="Agent for handling app requests.",
    idleSessionTTLInSeconds=1800,
    foundationModel="anthropic.claude-v2:1",
    instruction=silo_agent_instruction,
)

In [ ]:
silo_agent_id = silo_agent_response['agent']['agentId']
silo_agent_id

### Create Agent Action Group


In [ ]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
silo_agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=silo_agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': silo_lambda_function['FunctionArn']
    },
    actionGroupName='SiloAppServicesActionGroup', 
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description='Actions for creating product, get products, get a product, create order and get orders'
)

In [ ]:
silo_agent_action_group_response

### Allowing Agent to invoke Action Group Lambda


In [ ]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=silo_agent_lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{silo_agent_id}",
)

### Preparing Agent


In [ ]:
silo_agent_prepare = bedrock_agent_client.prepare_agent(agentId=silo_agent_id)
silo_agent_prepare

### Create Agent alias


In [ ]:
# Pause to make sure agent is prepared
time.sleep(30)
silo_agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=silo_agent_id,
    agentAliasName=silo_agent_alias_name
)

In [ ]:
silo_agent_alias

## Invoke Agent


In [ ]:
def invoke_silo_agent(tenantId, task_input_text):
    silo_agent_alias_id = silo_agent_alias['agentAlias']['agentAliasId']

    ## create a random id for session initiator id
    session_id:str = str(uuid.uuid1())
    enable_trace:bool = True
    end_session:bool = False
    
    silo_agentResponse = bedrock_agent_runtime_client.invoke_agent(
    sessionState={
        'sessionAttributes': {
            'tenantId': tenantId,
            
        }
    },
    inputText=task_input_text,
    agentId=silo_agent_id,
    agentAliasId=silo_agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
    )

    logger.info(pprint.pprint(silo_agentResponse))
    capture_metrics(silo_agentResponse, tenantId)
    

### Create data for tenant3

In [ ]:
# invoke the agent API
tenantId='tenant3'
task_input_text = 'create two products product5 and product6 with prices 50 and 60 respectively' 
invoke_silo_agent(tenantId, task_input_text)

### Get all products of tenant3

In [ ]:
tenantId='tenant3'
task_input_text = 'get all products details and display' 
invoke_silo_agent(tenantId, task_input_text)

## Thank You
Note: Please delete resources created through this notebook if you don't want to incur any costs.
